In [25]:
import torch
import torch.cuda
import torch.nn as nn
import dataset
from dataset import MyDataset
import converter
from torch.utils.data import DataLoader
import os
import random
import json
import model
import torch.nn.functional as F

In [26]:
hyperparameters = {
    "lr": 0.001,
    "batch_size": 50,
    "num_epochs": 10,
    "manualSeed":1234,
    "alphabet":'0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ',
    "nhidden":256,
    "pretrained":'best_model2.pth',
    "optimizer":"Adam",
    "beta1":0.6,
    "weight_decay":0.01,
    "momentum":0.99
}
file_path = 'hyperparameters.json'
with open(file_path, 'w') as json_file:
    json.dump(hyperparameters, json_file, indent=4)
with open(file_path, 'r') as json_file:
    hyperparameters = json.load(json_file)



train_dataset = MyDataset(0)
test_dataset = MyDataset(1)
val_dataset = MyDataset(2)
train_loader = DataLoader(train_dataset,batch_size=hyperparameters["batch_size"],shuffle=True,drop_last=True,pin_memory=True)
test_dataset = DataLoader(test_dataset,batch_size=hyperparameters["batch_size"],shuffle=True,drop_last=True,pin_memory=True)
val_dataset = DataLoader(val_dataset,batch_size=hyperparameters["batch_size"],shuffle=True,drop_last=True,pin_memory=True)

nclass = len(hyperparameters["alphabet"])+1
converter = converter.strLabelConverter(hyperparameters["alphabet"])
criterion = nn.CTCLoss(blank= 0,reduction='mean')
print(converter.alphabet)
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv') != -1:
#         m.weight.data.normal_(0.0, 0.0)
#     elif classname.find('BatchNorm') != -1:
#         m.weight.data.normal_(0.0, 0.0)
#         m.bias.data.fill_(0)
def weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        # 使用 Xavier 初始化方法初始化卷积层权重
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)  # 可选：初始化偏置为 0
    elif isinstance(m, nn.BatchNorm2d):
        # 使用正态分布初始化 BatchNorm 层权重和偏置
        nn.init.normal_(m.weight, mean=0, std=1.0)
        nn.init.constant_(m.bias, 0.0)

model = model.CRNN(imgH=32,nclass=nclass,nh=hyperparameters["nhidden"],nc=1,leakyRelu=True)

#model.apply(weights_init)

if hyperparameters["pretrained"] != '':
    print('loading pretrained model from %s' %  hyperparameters["pretrained"])
    model.load_state_dict(torch.load(hyperparameters["pretrained"]))
print(model)

if hyperparameters["optimizer"]=="Adam":
    optimizer = torch.optim.Adam(model.parameters(),lr = hyperparameters["lr"],betas=(hyperparameters["beta1"],0.999),weight_decay=hyperparameters["weight_decay"])
elif hyperparameters["optimizer"]=="SGD":
    optimizer = torch.optim.SGD(model.parameters(),lr = hyperparameters["lr"],momentum=hyperparameters["momentum"],weight_decay=hyperparameters["weight_decay"])
elif hyperparameters["optimizer"]=="Adadelta":
    optimizer = torch.optim.Adadelta(model.parameters(),lr = hyperparameters["lr"],weight_decay=hyperparameters["weight_decay"])
else: optimizer = torch.optim.RMSprop(model.parameters(), lr=hyperparameters["lr"],momentum=hyperparameters["momentum"])


class Trainer():
    def __init__(self,model,criterion,optimizer):
        self.model = model.to('cuda')
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                              milestones=[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
                                                              gamma=0.65)
    def train(self,epoch):
        i = 0
        j = 1
        while j:
            for data in train_loader:
                i = i+1
                if i>epoch:
                    j = j-1
                    break

                image,label = data
                image = image.unsqueeze(1).to('cuda')
                #print(label)
                text,length = converter.encode(label)
                text_tensor = torch.tensor(text).reshape(-1,6).to('cuda')#32*6
                preds  = self.model(image)#shape is 32*26*37
                preds = F.log_softmax(preds, dim=2)
                input_lengths = torch.full((preds.size(1),), preds.size(0), dtype=torch.long).to('cuda')  # 假设输入序列长度相同，为 preds 的时间步数
                target_lengths = torch.full((text_tensor.size(0),), text_tensor.size(1), dtype=torch.long).to('cuda') # 假设目标序列长度相同，为 text 的字符数
                #print(f'pred{preds},text_tensor{text_tensor},inputleng{input_lengths},target{target_lengths}')
                print(f'{preds.max(2)[1]}')
                loss = self.criterion(preds, text_tensor,input_lengths,target_lengths)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                
                if i %1==0:print(f'epoch:{i},loss:{loss}')

                if i %10 == 0: 
                    torch.save(self.model.state_dict(), 'best_model3.pth')
                    print("Your model has been saved")


if __name__ == '__main__':
    Trainer = Trainer(model = model,criterion=criterion,optimizer=optimizer)
    Trainer.train(100000)

0123456789abcdefghijklmnopqrstuvwxyz-
loading pretrained model from best_model2.pth
CRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu0): LeakyReLU(negative_slope=0.2, inplace=True)
    (pooling0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): LeakyReLU(negative_slope=0.2, inplace=True)
    (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): LeakyReLU(negative_slope=0.2, inplace=True)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): LeakyReLU(negative_slope=0.2, inplace=True)
    (pooling2): MaxPool2d(kernel_size=(2, 2), stride=(2, 1),

C:\Users\HONOR\AppData\Local\Temp\ipykernel_21568\2117764071.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_tensor = torch.tensor(text).reshape(-1,6).to('cuda')#32*6


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')
epoch:2,loss:7.475076675415039
torch.Size([50, 512, 1, 26])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')
epoch:3,loss:3.8379604816436768
torch.Size([50, 512, 1, 26])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')
epoch:4,loss:4.589965343475342
torch.Size([50, 512, 1, 26])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,

KeyboardInterrupt: 